# LlamaParse - Fast checking Insurance Contract for Coverage

<a href="https://colab.research.google.com/github/run-llama/llama_cloud_services/blob/main/examples/parse/demo_insurance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we will look at how LlamaParse can be used to extract structured coverage information from an insurance policy.

Status:
| Last Executed | Version | State      |
|---------------|---------|------------|
| Aug-19-2025   | 0.6.61  | Deprecated |

## Installation of required packages

In [ ]:
%pip install "llama-index>=0.13.0<0.14.0" llama-parse

## Download an insurance policy from IRDAI

The Insurance Regulatory and Development Authority of India (IRDAI) maintains a great resource: https://policyholder.gov.in/web/guest/non-life-insurance-products where all insurance policies available in India are publicly available for download! Let's download a complex health insurance policy as an example.

In [ ]:
!wget "https://policyholder.gov.in/documents/37343/931203/NBHTGBP22011V012223.pdf/c392bcc1-f6a8-cadd-ab84-495b3273d2c3?version=1.0&t=1669350459879&download=true" -O "./policy.pdf"

## Initializing LlamaIndex and LlamaParse

In [ ]:
import os

os.environ["LLAMA_CLOUD_API_KEY"] = "llx-..."
os.environ["OPENAI_API_KEY"] = "sk-..."

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.core import Settings

# for the purpose of this example, we will use the small model embedding and gpt3.5
embed_model = OpenAIEmbedding(model="text-embedding-3-small")
llm = OpenAI(model="gpt-5-mini")

Settings.llm = llm
Settings.embed_model = embed_model

## Vanilla Approach - Parse the Policy with LlamaParse into Markdown

In [ ]:
from llama_cloud_services import LlamaParse

result = await LlamaParse().aparse("./policy.pdf")
documents = result.get_markdown_documents(split_by_page=True)

Started parsing the file under job_id 35052045-ce36-4343-9e7c-11e059a59cc2


In [ ]:
print(documents[0].text[0:1000])

Bupa niva Health Insurance

# 1. Preamble

This ‘Travel Infinity’ Policy is a contract of insurance between You and Us which is subject to payment of full premium in advance and the terms, conditions and exclusions of this Policy. Expense incurred outside the policy period will NOT be covered. Unutilized Sum Insured will expire at the end of policy year. All applicable benefits, details and limits are mentioned in your Certificate of insurance. We will cover only allopathic treatments in this policy.

# 2. Defined Terms

The terms listed below in this Section and used elsewhere in the Policy in Initial Capitals shall have the meaning set out against them in this Section.

# Standard Definitions

# 2.1

Accident or Accidental means sudden, unforeseen and involuntary event caused by external, visible and violent means.

# 2.2

Co-payment means a cost sharing requirement under a health insurance policy that provides that the policyholder/insured will bear a specified percentage of the adm

In [ ]:
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

### Querying the model for coverage

In [ ]:
query_1 = "My flight was delayed 8 hours and I paid $450, how much am I covered for?"

response_1 = await query_engine.aquery(query_1)
print(str(response_1))

I can’t give an exact dollar amount without the values shown on your Certificate of Insurance. How the claim would be settled:

1. First check that your policy’s required delay threshold is met (the policy only pays if the delay exceeds the number of hours shown on your Certificate). Also the insurer won’t pay if the delay was publicly known at least 6 hours before departure.

2. Find which benefit option applies on your Certificate: a fixed payment or reimbursement of actual alternate-travel cost.
   - If a fixed payment applies, you will receive the fixed sum listed on the Certificate (regardless of the $450 you paid), subject to the other conditions and any deductible shown.
   - If reimbursement applies, the insurer will reimburse up to the Sum Insured shown on the Certificate, but will first deduct any compensation paid by the airline or other sources and then apply the deductible.

3. Reimbursement formula (if reimbursement option applies):
   Payable = max(0, min(Sum Insured, Am

The information is split across the document which leads to retrieval issues. Let's try some parsing instructions to improve our result.

In [ ]:
result = await LlamaParse(
    result_type="markdown",
    system_prompt_append="""
This document is an insurance policy.
When a benefits/coverage/exlusion is describe in the document amend to it add a text in the following benefits string format (where coverage could be an exclusion).

For {nameofrisk} and in this condition {whenDoesThecoverageApply} the coverage is {coverageDescription}. 
                                        
If the document contain a benefits TABLE that describe coverage amounts, do not output it as a table, but instead as a list of benefits string.
                                       
""",
).aparse("./policy.pdf")

documents_with_instruction = result.get_markdown_documents(split_by_page=True)

Started parsing the file under job_id c89abe4b-0bb3-4e04-a37f-1da880392346
.

Let see how the 2 parsing compare (change target page to explore)

In [ ]:
target_page = 45

print(documents[target_page].text)
print("\n\n=========================================================\n\n")
print(documents_with_instruction[target_page].text)


Inpatient treatment

# Claim Form (filled and signed by the Insured)

# Hospital Daily Cash

# Release of Medical information Form (filled and signed by the Insured)

# Waiver of Deductible

# Original pathological and diagnostic reports, discharge summary indoor case papers (if any) and prescriptions issued by the treating Medical practitioner or Network Provider

# Adventure Sports Cover

# Home to Home Cover

# Extension to in-patient care

# Ambulance Charge

# Out-patient treatment

# Cancer Screening &#x26; Mammographic Examination

# New Born baby Cover

# Maternity

# Complete pre-existing disease cover

# Medical sum insured replenishment in case of hospitalization due to accident

# Waiver of sublimit for insured above 60 years of age

# Psychiatric Counseling

# Physiotherapy

# Terrorism cover

# Medical tele-consultation

# Medical evacuation

Medical reports and transportation details issued by the evacuation agency, prescriptions and medical report by the attending Medi

In [ ]:
instruction_index = VectorStoreIndex.from_documents(documents_with_instruction)
query_engine_instruction = instruction_index.as_query_engine()

## Comparing Instruction-Augmented Parsing vs. Vanilla Parsing

When we parse the document with natural language instructions to add context on insurance coverage, we are able to correctly answer a wide range of queries in our RAG pipeline. In contrast, a RAG pipeline built with the vanilla method is not able to answer these queries.

In [ ]:
query_1 = "My flight was delayed 8 hours and I paid $450, how much am I covered for?"

response_1 = await query_engine.aquery(query_1)
print("Vanilla:")
print(response_1)

print("With instructions:")
response_1_i = await query_engine_instruction.aquery(query_1)
print(response_1_i)

Vanilla:
I can’t give an exact payout without details from your Certificate of Insurance. What matters is which benefit applies and the certificate values. Here’s how to determine the amount and some examples:

What to check on your certificate (send these if you want a precise calculation)
- Which benefit is being used: Flight Delay (alternate travel booking reimbursement or fixed amount) or Trip Delay (fixed amount per block of hours).  
- The minimum delay threshold (the number of hours the delay must exceed).  
- Whether the policy pays reimbursement or a fixed amount (and, if fixed, the amount per block and length of each block).  
- Sum Insured / maximum limit for that benefit.  
- Deductible (amount you must absorb per claim).  
- Any compensation already paid by the airline or other source (this is deducted from the insurer’s payment).  
- Reason for the delay and whether it’s an excluded reason (e.g., delay was publicly known 6+ hours before departure).

How to calculate (gene

Looking at the policy it says in list I that one expense not covered is Baby food

In [ ]:
query_2 = "I just had a baby, is baby food covered?"

response_2 = await query_engine.aquery(query_2)
print("Vanilla:")
print(response_2)

print("With instructions:")
response_2_i = await query_engine_instruction.aquery(query_2)
print(response_2_i)

Vanilla:
No. Food and beverages (including baby food) are excluded as expenses not linked to treatment. The policy only covers medical treatment and specified newborn items (e.g., emergency inpatient care and vaccinations — vaccinations limited to USD 500) and explicitly excludes "baby charges" unless specifically indicated.
With instructions:
No. Baby food is not covered. The policy pays medical treatment expenses and expressly excludes items not linked to treatment (for example food and beverages), and it also lists "baby charges" as not payable unless specifically indicated.  

Newborn medical treatment and vaccinations can be covered under the newborn/maternity benefits (vaccination cover is limited and subject to the policy's special conditions, waiting periods and deductibles), so check your certificate of insurance for any specific limits or endorsements.


In [ ]:
query_3 = "How is gauze used in my operation covered?"

response_3 = await query_engine.aquery(query_3)
print("Vanilla:")
print(response_3)

print("With instructions:")
response_3_i = await query_engine_instruction.aquery(query_3)
print(response_3_i)

Vanilla:
Gauze (including gauze soft) used in your operation is included within the procedure charges. It is subsumed into the surgical/procedure fee and is not payable as a separate item.
With instructions:
Gauze used during your operation is included in the procedure charges. Its cost is subsumed into the procedure/surgical fee and will not be reimbursed as a separate line item.
